In [ ]:
import pandas as pd
import dateutil.parser as dateparser
import datetime


In [ ]:
# put filenames in as whichever made files you ran the original ipynb for
filenames = ["allen-p_sent.csv", "arnold-j_sent.csv"]

columns = ["uid", "sender", "recipient", "sendtime", "replytime", \
                           "replyspeed", "messagecontent", "foldername"]
data = pd.DataFrame(columns=columns)
for filename in filenames:
    df = pd.read_csv("processed_data/"+filename)
    data = pd.concat([data, df], ignore_index=True)
    # data.concat(df, ignore_index=True)
# data.set_index('uid', drop=True, inplace=True)
data = data.set_index('uid')
data
# data.size

In [ ]:
data = data[data['replytime'] != '-1']
data.head()

In [ ]:
# much of the code here follows this tutorial: https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

import re

# Remove punctuation
data['messagecontent_processed'] = \
data['messagecontent'].map(lambda x: re.sub('[,\.!?]', '', x))# Convert the titles to lowercase
data['messagecontent_processed'] = \
data['messagecontent_processed'].map(lambda x: x.lower())# Print out the first rows of papers
data['messagecontent_processed'].head()

data.head()

In [ ]:
import nltk
nltk.download()

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data_x = data.messagecontent_processed.values.tolist()
data_words = list(sent_to_words(data_x))# remove stop words
data_words = remove_stopwords(data_words)
data_words[:1][0].remove('message')
data_words[:1][0].remove('original')
data_words[:1][0].remove('sent')
print(data_words[:1][0][:10])

In [ ]:
data.recipient.unique()
data_dict = data.to_dict('index')
data_dict

analyzed_dict = {}
for key in data_dict.keys():
    row = data_dict[key]
    
    replyspeed = pd.Timedelta(row['replyspeed']).total_seconds()
    sender = row['sender']
    recipient = row['recipient']
    # replyspeed = row['replyspeed']
    content = row['messagecontent_processed']
    tagged = False
    topic = ''
    for each in data_words[:1][0][:10]:
        if each in content:
            topic = each
            tagged = True
            break
    if not tagged:
        topic = 'unknown'

    if(sender, recipient) in analyzed_dict:
        analyzed_dict[sender, recipient].append((topic, replyspeed))
    else:
        analyzed_dict.update({(sender, recipient) : [(topic, replyspeed)]})
    

print(len(analyzed_dict.keys()))
print(len(analyzed_dict.values()))
        

In [ ]:
new_df = pd.DataFrame.from_dict(analyzed_dict, orient='index')
headers = ['(sender,recipient)']
for i in range(0, 48): headers.append('(email, speed)' + str(i))
# # print(new_df.head())
new_df.to_csv("processed_data/analyzed_data.csv", header=headers)
new_df.head()